In [7]:
import folium
import requests
import json
import pandas as pd
from utils2 import lien_google

# Clé API
API_KEY = "m2Q6PLrDbFonmYwJT-IqBq6DmYX_MhZ_DV6Gnls4JTnj0qUtuNKnaEt48q0lr9mrZM-husmMgztJc4TKF9TbChpv7nfCKU5GYKH7AxT3rtWwSJBVBtR8RfZmihaSZ3Yx"

# URL de base
url = "https://api.yelp.com/v3/businesses/search"

# Paramètres de la requête
params = {
    "location": "Paris",  # Exemple : ville
    "term": "restaurants",  # Exemple : type d'entreprise
    "limit": 50,            # Nombre de résultats
    "sort_by": "best_match",  # Critères de tri
}

# En-têtes HTTP
headers = {
    "Authorization": f"Bearer {API_KEY}",  # Clé API
    "Accept": "application/json"           # Format attendu
}

# Envoi de la requête GET
r = requests.get(url, headers=headers, params=params)

# Vérification du statut de la requête
if r.status_code == 200:
    data = r.json()
    businesses = data['businesses']  # Récupérer les informations sur les entreprises (restaurants)

    # Créer un DataFrame à partir des données
    restaurants_data = []
    for business in businesses:
        name = business['name']
        latitude = business['coordinates']['latitude']
        longitude = business['coordinates']['longitude']
        address = business['location']['address1'] if 'address1' in business['location'] else "N/A"
        url = business['url']
        lienG = lien_google(name, business["location"].get("city"))
        restaurants_data.append({'Name': name, 'Latitude': latitude, 'Longitude': longitude, 'Address': address, 'URL':url, 'URL2':lienG})

    df_restaurants = pd.DataFrame(restaurants_data)

    # Initialiser la carte centrée sur Paris (vous pouvez ajuster selon votre besoin)
    map_center = [48.8566, 2.3522]  # Coordonnées centrées sur Paris
    m = folium.Map(location=map_center, zoom_start=13)
   
       
     # Ajouter des marqueurs pour chaque restaurant
    for idx, row in df_restaurants.iterrows():
        popup_content = f"""
        <b>{row['Name']}</b><br>
        {row['Address']}<br>
        <a href="{row['URL']}" target="_blank">Lien vers Yelp</a><br>
        <a href="{row['URL2']}" target="_blank">Lien vers Google</a>
        """
        folium.Marker(
            [row['Latitude'], row['Longitude']],
            popup=folium.Popup(popup_content, max_width=300),
            icon=folium.Icon(icon='cutlery', color='blue')
        ).add_to(m)

    # Sauvegarder la carte sous forme de fichier HTML
    m.save("restaurants_map.html")
    print("Carte sauvegardée sous 'restaurants_map.html'.")
else:
    print("Erreur lors de la récupération des données:", r.status_code)
        



Carte sauvegardée sous 'restaurants_map.html'.


In [6]:
m